# Анкета 2.1
### Библиотеки

In [ ]:
from docxtpl import DocxTemplate, InlineImage
from docx.enum.dml import MSO_THEME_COLOR_INDEX
from docx.shared import Cm
import numpy as np
import pandas as pd 
from math import pi
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
import requests
import re
from string import ascii_letters
import ast
import docx2txt
import json
import docx
import os
import time
import datetime
import PySimpleGUI as sg
from string import punctuation
from pymystem3 import Mystem
from stop_words import get_stop_words
from matplotlib.patches import Circle, RegularPolygon
from matplotlib.path import Path
from matplotlib.projections.polar import PolarAxes
from matplotlib.projections import register_projection
from matplotlib.spines import Spine
from matplotlib.transforms import Affine2D
from matplotlib.ticker import FixedLocator
import matplotlib.pyplot as plt
from tqdm import tqdm

### Диск

In [ ]:
gauth = GoogleAuth()
# Try to load saved client credentials
gauth.LoadCredentialsFile("mycreds.txt")
if gauth.credentials is None:
    # Authenticate if they're not there
    gauth.LocalWebserverAuth()
elif gauth.access_token_expired:
    # Refresh them if expired
    gauth.Refresh()
else:
    # Initialize the saved creds
    gauth.Authorize()
# Save the current credentials to a file
gauth.SaveCredentialsFile("mycreds.txt")
drive = GoogleDrive(gauth)

### Засекаем время загрузки данных

In [ ]:
start_time = time.time()

### Все изображения на диске

In [ ]:
file_list = drive.ListFile({'q': "mimeType='image/jpeg' or mimeType='image/png'"}).GetList()
files = {'name': [], 'id': []}
for file in file_list:
    files['name'].append(file['title'])
    files['id'].append(file['id'])
images = pd.DataFrame(files)
images.head()

,name,id
0,Screenshot_2_tilda11057738.jpg,19ZXW5oa5t6l2eSHPuL3DKRBgAyqg6IjS
1,1648740337462_tilda11027987.jpg,1OXk0YiHY8Il7wuO7VeKFebl2xq3w2GCb
2,6W5A9558.jpg,1ZDRlL6DEdL_HitWgZLfryBDjl1BHAOS7
3,6W5A9506.jpg,1l3QEe1ApBalqm7j3j2xMHno7nwM7b3Gp
4,6W5A9502.jpg,1jjNUP7TtTshGC-oyzgdIkX8ZBoROUi0j


### Поиск ссылки в html коде

In [ ]:
def get_name(text):
    return re.findall(r'>(.+jpeg|.+.jpg|.+.png)', text, re.I)[0]

### Вставка картинки

In [ ]:
def screen_update(key, email, image_name, context, doc, size):
    if len(max(df[df['ma_email'] == email][key])) > 1:
        html = requests.get(max(df[df['ma_email'] == email][key])).text
        filename = get_name(html)
        file_id = images[images['name'] == filename]['id'].values[0]
        image = drive.CreateFile({'id': file_id})
        image.GetContentFile(image_name)
        return InlineImage(doc, image_name, Cm(size))
        #context[key] = InlineImage(doc, image_name, Cm(size))
    else:
        return 'пропуск'

### ID папки с таблицами и видеовопросами

In [ ]:
tables_root_id = drive.ListFile({'q': "title = 'Таблицы -> Анкеты'"}).GetList()[0]['id']
videoquest_folder_names = ['Вопрос от Катерины Ходуновой', 'Вопрос от Олега Лысака']
videoquest_root_ids = [drive.ListFile({'q': "title = '{}'".format(name)}).GetList()[0]['id'] for name in videoquest_folder_names]

### Все таблицы с данными

In [ ]:
tables = []
file_list = drive.ListFile({'q': " '%s' in parents and trashed = FALSE" % tables_root_id}).GetList()
for file in file_list:
    if file['title'] == 'Шаблон анкеты v.2.1.docx':
        print(file['title'])
        file.GetContentFile(file['title'])
        template_a = file['title']
    if file['title'] == 'Шаблон анкеты v.2.1a.docx':
        print(file['title'])
        file.GetContentFile(file['title'])
        template = file['title']
    if file['mimeType'] == 'image/png':
        print(file['title'])
        file.GetContentFile(file['title'])
        photo_default = file['title']
    if file['mimeType'] == 'application/vnd.google-apps.spreadsheet':
        file.GetContentFile(file['title'] + '.xlsx', mimetype = 'application/vnd.openxmlformats-officedocument.spreadsheetml.sheet')
        tables.append(file['title'] + '.xlsx')
    if file['mimeType'] == 'application/vnd.openxmlformats-officedocument.spreadsheetml.sheet':
        #if file['title'] == 'Тестирование. 21 вопрос.xlsx':
          #  file.GetContentFile(file['title'])
            #tables.append(file['title'])
        if file['title'] == 'Теория технологического предпринимательства - тест.xlsx':
            file.GetContentFile(file['title'])
            tables.append(file['title'])
    if file['mimeType'] == 'text/csv':
        file.GetContentFile(file['title'])
        tables.append(file['title'])
for file_id in videoquest_root_ids:
    file = drive.ListFile({'q': " '%s' in parents and trashed = FALSE" % file_id}).GetList()[0]
    file.GetContentFile(file['title'])
    tables.append(file['title'])
tables

Шаблон отчета v.5.0a.docx
Шаблон отчета v.5.0.docx
photo_default.png


['Cases.xlsx',
 'Game.xlsx',
 'Nice2MeetYou.xlsx',
 'Knowledge.xlsx',
 'Теория технологического предпринимательства - тест.xlsx',
 'QuestionsFromStudents.xlsx',
 'TestOpinion.xlsx',
 'Вопрос+от+Катерины+Ходуновой.xlsx',
 'Вопрос+от+Олега+Лысака.xlsx']

### Таблицы к DataFrame

In [ ]:
dfs = []
for table in tables:
    df = pd.read_excel(table, dtype = str, keep_default_na = False)
    if 'Потрачено времени' in df.columns:
        df = df.iloc[1:]
        df['Процент правильных ответов (%)'] = df['Процент правильных ответов (%)'].apply(float).round().apply(int).apply(str)
        df.rename(columns = {'Потрачено времени': 'time', 'Процент правильных ответов (%)': 'score_test'}, inplace = True)
    df.columns = [col.replace(' ', '').replace('/', '').replace('.', '').lower() for col in df.columns]
    dfs.append(df)

### Получение имен, городов

In [ ]:
index_table = [tab for tab in tables if tab.startswith('Nice')][0]
df0 = pd.read_excel(index_table, dtype = str, keep_default_na = False)
df0['gorodprojivaniya'].replace('/', ';', regex=True, inplace=True)
df0['ma_email'] = df0['ma_email'].str.lower()
emails = df0['ma_email'][:].unique()
emails

array(['bgdshv.a@gmail.com', 'elashkina.anna@gmail.com',
       'fedoseev.is@enrubattery.ru', ..., 'polly.2003t@gmail.com',
       'alina.kkurbatova@yandex.ru', 'danil.bragin.2020@gmail.com'],
      dtype=object)

### Создание папок

In [ ]:
try:
    os.mkdir('Вывод')
except:
    pass
for ref in df0['referer'].unique():
    try:
        os.mkdir('Вывод/' + re.findall('ship-(.+)/test', ref)[0])
    except:
        pass

### ID папки вывода

In [ ]:
folder_root_id = drive.ListFile({'q': "title = 'Поиск по кадровой базе данных'"}).GetList()[0]['id']

### Существующие папки на диске

In [ ]:
folder_names = {}
folder_files = drive.ListFile({'q': " '%s' in parents" % folder_root_id}).GetList()
for file in folder_files:
    folder_names[file['title']] = file['id']
folder_names

{'vyatgu(2)': '1qyXHj4rOVaJO9iRBC0skliMcvv1Yrhml',
 'vgtu': '1tFCCehFEkEAqogxXEdkYgjfrdEmeMq5i',
 'test2022': '1JO5S6PnGa6F9RYLVnMV4oEIqG0wjYUEN',
 'tyumgu': '14d6F-Y9fvmik4k6mPcG5BQ_XAUIFzWT_',
 'tusur': '1gQPo1Pk2s3iVYDLNHt7ElzChvkv39Ps-',
 'urfu': '1VnvP5paZ7ZJafSOEZ3xDRXpa-7nUHASf',
 'rut': '1THw6Jthyeign82vUqPZzdF6YTDokEvqb',
 'vyatgu': '15kTG-EgYjWOx4ojbgQwlIZFHUTIdS0_8',
 'test1': '13l-4GiP-qybRiQpJc4INov36qoRKPa7Q',
 'tgu': '1nlGMSg83F1KCHZ5WiGpA1qiMAefnd4AN',
 'mfti': '1z25WyFW_i20KQLetk7l2EdGC-5WA95kZ',
 'mfti2021': '16ZARQzkvqTQY2qT5JJQurC7YbBF4wYRy',
 'omgtu-2021': '1MTjaF-_IEHY0bGmh6V4xiIE7uU2b9hfS',
 'startup': '19xMsXa_he8vqHSd6ki6AXZUXSjoG1xWj',
 'Первичный балл 2021-11-22 МФТИ.docx': '1la_zyT1sEfNqFu1HJDO3s6QDgGQMmEKh',
 'Первичный балл 2021-11-22 Омск.docx': '1hrvZV7YKOTFlwY6IAFGI4g2PlVA2PEjj',
 'Первичный балл Нальчик.docx': '1J9Nk3U6_SWDv_0VRU0Kv9lwFr9oo3z7R',
 'Первичный балл НГТУ.docx': '17xqp3_YFDfRIwPnE1mPUcyVj0gnhncQu',
 'ngtu': '1RCiIzxzd5szPURTHWhp9tUm_IUIbD9

### Создание папок на диске

In [ ]:
for ref in df0['referer'].unique():
    try:
        folder_name = re.findall('ship-(.+)/test', ref)[0]
        if folder_name not in folder_names.keys():
            print(folder_name)
            file = drive.CreateFile({'title': folder_name, 
                "parents": [{"id": folder_root_id}], 
                "mimeType": "application/vnd.google-apps.folder"})
            file.Upload()
            folder_names[file['title']] = file['id']
    except:
        pass

## Игры ПКПК

### Все досье на диске

In [ ]:
file_list = drive.ListFile({'q': "title contains '_игрок_' and title contains '.html'"}).GetList()
files = {'name': [], 'id': [], 'parent_id': []}
for file in file_list:
    files['name'].append(file['title'])
    files['id'].append(file['id'])
    files['parent_id'].append(file['parents'][0]['id'])
docs = pd.DataFrame(files, dtype = str)
docs.head()

,name,id,parent_id
0,_игрок_Шахвардян Ева.html,1ySqnM652mqw-KNXOtwXmpSrdZF8ICKAj,1hYWnZbcKj4StihWMStY6uSUqxYqvg5_m
1,_игрок_Шевченко Галина.html,1zK75L2H_iONqoOAklKPjSgqIOsoQkpdc,1hYWnZbcKj4StihWMStY6uSUqxYqvg5_m
2,_игрок_Яковлев Дмитрий.html,12DZj_-0GUaBjYBdGnlALNZQclqYm36CV,1hYWnZbcKj4StihWMStY6uSUqxYqvg5_m
3,_игрок_Шандыба Никита.html,1qyB9Y9ysfNLrwlPctv1B91E5D0q9OqQL,1hYWnZbcKj4StihWMStY6uSUqxYqvg5_m
4,_игрок_Шипулин Александр.html,1nnvOppDIVcA_wb8zB-7eIqao7xSO0ENC,1hYWnZbcKj4StihWMStY6uSUqxYqvg5_m


### ID корневой папки

In [ ]:
#root_id = drive.ListFile({'q': "title = 'OBSERVER'"}).GetList()[0]['id']

### Й к одной кодировке

In [ ]:
docs['name'] = [doc.replace('й', 'й') for doc in docs['name']]

### 0 < латинских символов < все ?

In [ ]:
def eng_char(nickname):
    return (any(map(lambda c: c in ascii_letters, nickname)) and not all(map(lambda c: c in ascii_letters + ' ', nickname)))

### Замена латиницы на кириллицу

In [ ]:
def replace(str):
    str = str.replace('e', '.+').replace('E', '.+').replace('T', '.+').replace('y', '.+').replace('o', '.+').replace('O', '.+').replace('p', '.+').replace('P', '.+').replace('M', '.+')
    str = str.replace('a', '.+').replace('A', '.+').replace('K', '.+').replace('x', '.+').replace('X', '.+').replace('c', '.+').replace('C', '.+').replace('B', '.+').lower()
    return(str)

### Поиск папки с досье

In [ ]:
def up_to_root(last, name, root_id):
    result = []
    if eng_char(last) or eng_char(name):
        last, name = replace(last), replace(name)
        doc_names = [doc for doc in docs['name'] if re.findall(last, doc.lower()) and re.findall(name, doc.lower())]
    else:
        doc_names = [doc for doc in docs['name'] if last.lower() in doc.lower() and name.lower() in doc.lower()]
    doc_list = docs[docs['name'].isin(doc_names)]
    for id in doc_list['id']:
        id_now = doc_list[doc_list['id'] == id]['parent_id'].values[0]
        file_now = drive.CreateFile({'id': id_now})
        parent_id_now = file_now['parents'][0]['id']
        while parent_id_now != root_id:
            id_now = parent_id_now
            file_now = drive.CreateFile({'id': id_now})
            parent_id_now = file_now['parents'][0]['id']
        result.append(file_now['title'])
    return(result)

### Колонка почты

In [ ]:
def email_col(df):
    return max([col for col in df.columns if 'mail' in col], key=len)

## Первичный балл

### Словарь баллов

In [ ]:
primary_keys_dict = {'chtoudivilomaterials': 5, 'chtorazozlilomaterials': 5,
                                'vopros1': 1, 'vopros2': 1, 'vopros3': 1,
                                'q1': 4, 'q2': 4,
                                'question': 3}
primary_cases_list = ['case1', 'case2', 'case3', 'case4']

### Токенизация

In [ ]:
# Lemmatizer and stopwords
mystem = Mystem() 
stop_words = set(get_stop_words('russian'))

# Preprocess function
def preprocess_text(text):
    tokens = mystem.lemmatize(re.sub(r'\d+', '', text.lower()))
    tokens = [token for token in tokens if token not in stop_words
              and token != " "
              and token.strip() not in punctuation]
    
    return list(dict.fromkeys(tokens))

def case_counter(text, case):
    tokens = preprocess_text(text)
    
    return sum(word in dict_cases[case] for word in tokens)

### Словари кейсов

In [ ]:
dict_cases = {
    'case1': "представлять предприниматель компания заниматься разработка высокоточный станок производство пробный партия необходимо купить специальный клей исследовать возможный предложение интернет понимать поставка организовывать германия необходимый количество стоять евро самостоятельно приходиться сделать следующий заплатить закупка доставка налог пошлина потратить рабочий оформление документ ждать счастие инженер отец жить близкий собираться навещать сын заказывать домой привозить багаж неделя действовать официальный воспользоваться удачный стечение обстоятельство написать цепочка рассуждение подробно аргументировать ответ".split(' '),
    'case2': "представлять коммерческий директор компания разработка отопительный котел новый поколение месяц выпускать прототип показывать заказчик огромный строительный фирма оставаться довольный сделать следующий предложение готовый купить экземпляр пробный запускать достраивать здание проба продлиться оказываться годный сразу партия знать рынок конкурент каков особенность продукция данный момент участвовать испытание соглашаться отдавать начинать оформлять тз производство штука контрактный главный разработчик сообщать находить решение повышать производительность переделывать опытный образец конструкторский бюро проверять тестовый стенд число поменять короб помещаться отправлять устаревать конструкция инженер говорить срочно предлагать вернуть тестирование утверждать  % ,  вариант причина выбирать стоять вопрос обращаться прерывать середина путь технический характеристика продукт рискнуть потратить деньги подготовка документация улучшать точно скоро хороший примета складываться ситуация написать цепочка рассуждение подробно аргументировать ответ".split(' '),
    'case3': "представлять молодой предприниматель открывать небольшой стартап партнерство одногруппник разрабатывать крутой приложение смартфон уверять взлетать пользоваться огромный спрос очередной этап разработка понимать свой деньги хватать нужный инвестиция нанимать дополнительный разработчик дизайнер тестировщик покупать лицензия облако место надежный сервер посчитать близкий находить млн рубль неделя поиск приходить кредитный предложение банк предлагать вполне сносный условие взять льготный годовой получать перевод завтра отвечать венчурный фонд mobileventures согласный проинвестировать транш входить доля ждать результат месяц возможный следующий раунд переговоры партнер встречаться двоюродный дядя рассказывать идея понравиться склон скопить приличный сумма заниматься поставка продукт питание из-за рубеж готовый вкладывать единственный поставлять держать курс дело принятие решение интересно следить строиться новый технология поступать источник вообще написать цепочка рассуждение подробно аргументировать ответ".split(' '),
    'case4': "представлять начальник департамент инновация крупный корпорация подобный ibm усиливать инновационность деятельность собственно продукт достаточно большой бюджет поставлять амбициозный задача ряд проектный сессия вариант деньги выделять хватить начинать инвестировать самостоятельно фонд молодой стартапа нужный направление ожидать выходить улучшать повышать эффективность работа самый инвестиция предполагать получение доля компания должный возвращать последствие либо полностью выкупать процент успешность оценивать предлагать сотрудник сумма экономия приносить идея обязывать гарантировать рассмотрение поступать обоснованный ответ случай отказ принимать  –  таков корпоративный этика решение совещание совет директор".split(' ')
    }

### Словари новеллы

In [ ]:
q_scored = ['q23', 'q50', 'q34', 'q41', 'q48', 'q62', 'q70']
q_translated = ['q10', 'q18', 'q33', 'q40', 'q45', 'q48', 'q59', 'q60', 'q61', 'q70']

### ID папки с новеллой

In [ ]:
novella_root_id = drive.ListFile({'q': "title = 'НовеллаБД'"}).GetList()[0]['id']

### Загружаем данные о новелле

In [ ]:
file_list = drive.ListFile({'q': " '%s' in parents and trashed = FALSE" % novella_root_id}).GetList()
for file in file_list:
    if file['mimeType'] == 'application/vnd.openxmlformats-officedocument.spreadsheetml.sheet':
        file.GetContentFile(file['title'])
        rules = pd.read_excel(file['title'])
"""
    if file['mimeType'] == 'text/csv':
        file.GetContentFile(file['title'])
        data = pd.read_csv(file['title'], sep = ";", encoding="windows-1251", quotechar = '"')
"""
url = 'https://10education.ru'
r = requests.get(url)
first_list = r.text.split('<br />')
index_pass = first_list[0].index('/html')
first_list[0] = first_list[0][index_pass+10:]
second_list = [string.split(' - ') for string in first_list]
df = pd.DataFrame(second_list)
columns_names = {0: 'username', 1: 'email', 2: 'password', 3: 'history_new'}
data = df.rename(columns=columns_names)
data['ma_email'] = data['email']
data = data[:-1]
print('Данные получены: '+url)

Данные получены: https://10education.ru


### Добавляем данные о новелле

In [ ]:
def novella_insert(email, data, rules):
    if data[data['email'] == email]['history_new'].to_list() != []:
        h_str = data[data['email'] == email]['history_new'].to_list()[0]
        h_json = json.loads(h_str)
        h_list = [rec for rec in h_json.values() if rec != {}]
        if h_list == []:
            return('-', '-', '-', '-', '-')
        h_record = max(h_list, key=len)
        novella_score = len(set(h_record.keys()).intersection(q_scored)) * 5
        if 'q19' in h_record.keys() and h_record['q19'] == 'a1':
            novella_score += 5
        if 'q37' in h_record.keys() and h_record['q37'] == 'a2':
            novella_score += 5
        if "Использовать старые данные" in h_record.values():
            novella_score += 5
        novella_translated = []
        novella_last = 'пропуск'
        novella_pizza = np.array([0, 0, 0, 0, 0, 0, 0, 0])

        for key, value in h_record.items():
            if key in rules['Вопрос'].to_list() and value in rules.columns.to_list():
                rules_row = (rules[rules['Вопрос'] == key])
                value_new = rules_row[value+'_axial_value'].to_list()[0]
                try:
                    novella_pizza += np.array(ast.literal_eval(value_new))
                except:
                    pass
                if key in q_translated:
                    value_translated = rules_row[value+'_i'].to_list()[0]
                    novella_translated.append(value_translated)
        novella_translated = " ".join(novella_translated)

        try:
            for pair in list(h_record.items())[::-1]:
                if pair[0].startswith('q'):
                    key_last, value_last = pair
                    rules_row = (rules[rules['Вопрос'] == key_last])
                    key_new = rules_row['Перевод'].to_list()[0]
                    value_new = rules_row[value_last+'_i'].to_list()[0]
                    novella_last = value_new
                    break
        except:
            pass

        if 'q21' in h_record.keys():
            novella_branch = 'Корпоративная'
        elif 'q49' in h_record.keys():
            novella_branch = 'Торговая'
        else:
            novella_branch = '-'

        return(novella_score, novella_translated, novella_last, novella_branch, novella_pizza)
    else:
        return('-', '-', '-', '-', '-')

### Пицца - граф

In [ ]:
def radar_factory(num_vars, frame='circle'):
    """Create a radar chart with `num_vars` axes.

    This function creates a RadarAxes projection and registers it.

    Parameters
    ----------
    num_vars : int
        Number of variables for radar chart.
    frame : {'circle' | 'polygon'}
        Shape of frame surrounding axes.

    """
    # calculate evenly-spaced axis angles
    theta = np.linspace(0, 2*np.pi, num_vars, endpoint=False)

    class RadarAxes(PolarAxes):

        name = 'radar'

        def __init__(self, *args, **kwargs):
            super().__init__(*args, **kwargs)
            # rotate plot such that the first axis is at the top
            self.set_theta_zero_location('N')

        def fill(self, *args, closed=True, **kwargs):
            """Override fill so that line is closed by default"""
            return super().fill(closed=closed, *args, **kwargs)

        def plot(self, *args, **kwargs):
            """Override plot so that line is closed by default"""
            lines = super().plot(*args, **kwargs)
            for line in lines:
                self._close_line(line)

        def _close_line(self, line):
            x, y = line.get_data()
            # FIXME: markers at x[0], y[0] get doubled-up
            if x[0] != x[-1]:
                x = np.concatenate((x, [x[0]]))
                y = np.concatenate((y, [y[0]]))
                line.set_data(x, y)

        def set_varlabels(self, labels):
            self.set_thetagrids(np.degrees(theta), labels)

        def _gen_axes_patch(self):
            # The Axes patch must be centered at (0.5, 0.5) and of radius 0.5
            # in axes coordinates.
            if frame == 'circle':
                return Circle((0.5, 0.5), 0.5)
            elif frame == 'polygon':
                return Circle((0.5, 0.5), 0.5)
            else:
                raise ValueError("unknown value for 'frame': %s" % frame)

        def draw(self, renderer):
            """ Draw. If frame is polygon, make gridlines polygon-shaped """
            if frame == 'polygon':
                gridlines = self.yaxis.get_gridlines()
                for gl in gridlines:
                    gl.get_path()._interpolation_steps = num_vars
            super().draw(renderer)

        def _gen_axes_spines(self):
            if frame == 'circle':
                return super()._gen_axes_spines()
            elif frame == 'polygon':
                # spine_type must be 'left'/'right'/'top'/'bottom'/'circle'.
                spine = Spine(axes=self,
                              spine_type='circle',
                              path=Path.unit_regular_polygon(num_vars))
                # unit_regular_polygon gives a polygon of radius 1 centered at
                # (0, 0) but we want a polygon of radius 0.5 centered at (0.5,
                # 0.5) in axes coordinates.
                spine.set_transform(Affine2D().rotate(pi/8).scale(0.5).translate(.5, .5)
                                    + self.transAxes)

                return {'polar': spine}
            else:
                raise ValueError("unknown value for 'frame': %s" % frame)

    register_projection(RadarAxes)
    return theta

### Вставка пиццы

In [ ]:
col1 = 'Дисконтирование\nустаревших\nрешений'
col2 = 'Давление на\nнезависимого\nпартнера'
col3 = 'Конструирование\n компаний'
col4 = 'Препарирование\nтехнологий'
col5 = 'Скрининг\nшансов'
col6 = 'Борьба за\nпривилегии'
col7 = 'Фиктивная\nдеятельность'
col8 = 'Отношения\nвместо дела'
zero = 0

def web(doc, numbers):
    dict1 = {col : zero for col in [col1, col2, col3, col4, col5, col6, col7, col8]}
    N = 8
    theta = radar_factory(8, frame='polygon')

    angles = np.linspace(0, 2 * pi, N, endpoint=False)
    angles_mids = angles + (angles[1] / 2)
    #spoke_labels = data.pop(0)
    #title, case_data = data[0]

    fig, ax = plt.subplots(figsize=(12, 8), subplot_kw=dict(projection='radar'))
    #fig.subplots_adjust(left=0)
    ax.xaxis.set_minor_locator(FixedLocator(angles))
    fig.subplots_adjust(top=0.85, bottom=0.05)
    plt.ylim([0, 10])

    ax.set_theta_offset(pi/8-pi/4)
    ax.set_theta_direction(1)
    ax.set_xticks(angles_mids)
    ax.set_xticklabels(dict1.keys())
    ax.grid(True, axis='x', which='minor')
    ax.grid(False, axis='x', which='major')
    ax.grid(True, axis='y', which='major')
    ax.set_rgrids([2,4,6,8,10], [])
    #ax.set_title(file_name,  position=(0.5, 1.1), ha='center')
    color1='springgreen'
    color2='seagreen'
    color3='tomato'
    color4='red'
    for i in range(6):
        ax.fill_between(angles[i:i+2], 0, numbers[i] , facecolor=color1, edgecolor=color2, linewidth=2)
    ax.fill_between(angles[5:7], 0, numbers[5], facecolor=color3, edgecolor=color4, linewidth=2)
    ax.fill_between(angles[6:8], 0, numbers[6], facecolor=color3, edgecolor=color4, linewidth=2)
    ax.fill_between([angles[7],(angles[7]+angles[1])], 0, numbers[7], facecolor=color3, edgecolor=color4, linewidth=2)

    for index, label in enumerate(ax.get_xticklabels()):
        label.set_weight("bold")
        label.set_fontsize(14)
        if index in [0, 1, 7]:
            label.set_horizontalalignment("left")
        elif index in [3, 4, 5]:
            label.set_horizontalalignment("right")

    plt.savefig('web.png', transparent=True)
    return InlineImage(doc, 'web.png', Cm(12))

### Подсчет баллов

In [ ]:
def primary_point(context):
    if 'score' in context:
        pts = 0.2 * max([int(num) for num in context['score'].split()])
    else:
        pts = 0
    for key, value in primary_keys_dict.items():
        if key in context.keys() and len(context[key]) > 10:
            pts += value
    for key in primary_cases_list:
        if key in context.keys():
            if len(context[key]) >= 50:
                pts += 1
            if len(context[key]) >= 300:
                pts += 2
            counter = case_counter(context[key], key)
            if counter >= 4:
                pts += 2
            if counter >= 6:
                pts += 1
            if counter >= 8:
                pts += 1
    if context['novel_score'] != '-':
        pts += context['novel_score']
        if pts > 100:
            pts = 100
            
    return round(pts)

### Подсчет баллов пицца

In [ ]:
cases_pizza_low = {'case1': np.array([0.5, 0, 0.5, 0, 1, 0, 0 ,0]), 'case2': np.array([0, 0.5, 0.5, 1.5, 0, 0, 0 ,0]), 'case3': np.array([0, 1, 0, 0, 0.5, 0, 0 ,0])}
cases_pizza_high = {'case1': np.array([1, 0, 1, 0, 2, 0, 0 ,0]), 'case2': np.array([0, 1, 1, 3, 0, 0, 0 ,0]), 'case3': np.array([0, 2, 0, 0, 1, 0, 0 ,0])}

def score_cases_pizza(context):
    score_cases = np.array([0, 0, 0, 0, 0, 0, 0 ,0])
    if 'score' in context:
        if int(context['score']) >= 90:
            score_pizza = np.array([0, 0, 2, 1, 0, 0, 0, 0])
        elif int(context['score']) >= 60:
            score_pizza = np.array([0, 0, 1, 0, 0, 0, 0, 0])
        else:
            score_pizza =  np.array([0, 0, 0, 0, 0, 0, 0 ,0])
        score_cases += score_pizza
    for key in cases_pizza_low.keys():
        pts =  0
        if key in context.keys():
            if len(context[key]) >= 50:
                pts += 1
            if len(context[key]) >= 300:
                pts += 2
            counter = case_counter(context[key], key)
            if counter >= 4:
                pts += 2
            if counter >= 6:
                pts += 1
            if counter >= 8:
                pts += 1
        if pts >= 6:
            score_case = cases_pizza_high[key]
        if pts >= 3:
            score_case = cases_pizza_low[key]
        else:
            score_case = np.array([0, 0, 0, 0, 0, 0, 0 ,0])
        score_cases = score_cases + score_case
       # print(key, score_case, score_cases)
            
    return score_cases

### Вставка ссылки

In [ ]:
def add_hyperlink(paragraph, text, url):
    # This gets access to the document.xml.rels file and gets a new relation id value
    part = paragraph.part
    r_id = part.relate_to(url, docx.opc.constants.RELATIONSHIP_TYPE.HYPERLINK, is_external=True)

    # Create the w:hyperlink tag and add needed values
    hyperlink = docx.oxml.shared.OxmlElement('w:hyperlink')
    hyperlink.set(docx.oxml.shared.qn('r:id'), r_id, )

    # Create a w:r element and a new w:rPr element
    new_run = docx.oxml.shared.OxmlElement('w:r')
    rPr = docx.oxml.shared.OxmlElement('w:rPr')

    # Join all the xml elements together add add the required text to the w:r element
    new_run.append(rPr)
    new_run.text = text
    hyperlink.append(new_run)

    # Create a new Run object and add the hyperlink into it
    r = paragraph.add_run ()
    r._r.append (hyperlink)

    # A workaround for the lack of a hyperlink style (doesn't go purple after using the link)
    # Delete this if using a template that has the hyperlink style in it
    r.font.color.theme_color = MSO_THEME_COLOR_INDEX.HYPERLINK
    r.font.underline = True

    return hyperlink

### Печать таблицы

In [ ]:
def print_df(df, filename):
    df.insert(0, "№", range(1, len(df) + 1))
    doc = docx.Document()
    t = doc.add_table(df.shape[0]+1, df.shape[1])
    for j in range(df.shape[-1]):
        t.cell(0,j).text = df.columns[j]
    for i in range(df.shape[0]):
        for j in range(df.shape[-1]):
            t.cell(i+1,j).text = str(df.values[i,j])
    doc.save(filename)

### Печать таблицы с ссылками

In [ ]:
def print_df_linked(filename_in, filename_out):
    document = docx.Document(filename_in)
    
    tabs = []
    for table in document.tables:
        dff = [['' for i in range(len(table.columns))] for j in range(len(table.rows))]
        for i, row in enumerate(table.rows):
            for j, cell in enumerate(row.cells):
                if cell.text:
                    dff[i][j] = cell.text
        tabs.append(pd.DataFrame(dff))
    df = tabs[0]
    df.columns = df.iloc[0]
    df = df[1:]
    try:
        cities = df['Город'].unique()
    except:
        return 0
    link_dict = {}
    for city in cities:
        city_id = drive.ListFile({'q': "title = '%s'" % city}).GetList()[0]['id']
        for file in drive.ListFile({'q': " '%s' in parents and trashed = FALSE" % city_id}).GetList():
            link_dict.update({file['title'][:-5]: file['alternateLink']})

    df['Ссылка'] = df['ФИО'].copy()
    links = df['ФИО'].apply(lambda x: link_dict[x] if x in link_dict else 'Error').to_list()
    doc = docx.Document()
    t = doc.add_table(df.shape[0]+1, df.shape[1])
    for j in range(df.shape[-1]):
        t.cell(0,j).text = df.columns[j]
    for i in range(df.shape[0]):
        for j in range(df.shape[-1]):
            if (j+1 == df.shape[-1]):
                t.cell(i+1,j).text = ''
                if links[i] != 'Error':
                    add_hyperlink(t.cell(i+1,j).paragraphs[0], 'Ссылка', links[i])
                else:
                    t.cell(i+1,j).text = 'Ошибка'
            else:
                t.cell(i+1,j).text = str(df.values[i,j])
    doc.save(filename_out)
    
    return 0

### Запись на диск

In [ ]:
def drive_write(file_name, parent_id, file_path, replace = True):
    if replace:
        for file in drive.ListFile({'q': "title = '%s' and '%s' in parents " % (file_name, parent_id)}).GetList():
            file.Delete()
    file = drive.CreateFile({
        'title': file_name, 
        "parents": [{"id": parent_id}]
        })
    file.SetContentFile(file_path)
    file.Upload()

## Вывод

### Засекаем время составления анкет

In [ ]:
df0['referer'].unique()

array(['http://techpreneur-school.ru/intro-concepts-of-techpreneurship-ekb/test/nice2meetyou',
       'https://techpreneur-school.ru/intro-concepts-of-techpreneurship-ekb/test/nice2meetyou',
       'https://techpreneur-school.ru/intro-concepts-of-techpreneurship-tmn/test/nice2meetyou',
       'https://techpreneur-school.ru/intro-concepts-of-techpreneurship-kemerovo/test/nice2meetyou',
       'https://techpreneur-school.ru/intro-concepts-of-techpreneurship-nn/test/nice2meetyou',
       'https://techpreneur-school.ru/intro-concepts-of-techpreneurship-rd/test/nice2meetyou',
       'https://techpreneur-school.ru/intro-concepts-of-techpreneurship-2035/test/nice2meetyou',
       'https://techpreneur-school.ru/intro-concepts-of-techpreneurship-nanort/test/nice2meetyou',
       'https://techpreneur-school.ru/intro-concepts-of-techpreneurship-technospark/test/nice2meetyou',
       'https://techpreneur-school.ru/intro-concepts-of-techpreneurship-tomsk/test/nice2meetyou',
       'https://techpren

### Выбираем ключ (внимание)

In [ ]:
key_short = 'urfu'
ma_emails = df0[df0['referer']=='https://techpreneur-school.ru/intro-concepts-of-techpreneurship-{}/test/nice2meetyou'.format(key_short)]['ma_email'].unique()
#ma_emails = ['bikaviktoria@mail.ru']
emails = [email for email in ma_emails if 'test' not in email]
emails

['nikita174trofimov@gmail.com',
 'dmitriy.kovalchuk.1913@mail.ru',
 'hoseernendes23@gmail.com',
 'yana.kozlyakova@mail.ru',
 'svetlakova.99@mail.ru',
 'i.a.kholmanskikh@mail.ru',
 'mb572867@gmail.com',
 'shadrina.irushka@gmail.com',
 'neganlife@gmail.com',
 'khabibova1723@mail.ru',
 'flep00@mail.ru',
 'byzova-01@bk.ru',
 'smiraboi@mail.ru',
 'oly-veronika@mail.ru',
 'ya.egor00100@yandex.ru',
 'savkiv01@mail.ru',
 'sergazinov.01@mail.ru',
 'sarser1308@gmail.com',
 'a.s.zakharova@urfu.ru',
 'jennifer.ranches2020@gmail.com',
 'pponia123@gmail.com',
 'vedrova.nastya@yandex.ru',
 'stevengnoto1994@gmail.com',
 'yuliana.shimova@mail.ru',
 'l.piskunova1909@mail.ru',
 'volpeumok@mail.ru',
 'kirill100.km12@gmail.com',
 'stasia.voronova@yandex.ru',
 'anton.korzh@urfu.me',
 'h2oartemian@gmail.com',
 'mapmonicahian@gmail.com',
 '412759139@qq.com',
 'boroffichento@gmail.com',
 'felix-dem@mail.ru',
 'egor.sikov14@gmail.com',
 'stalker15682@gmail.com',
 'zincenkosnezana82@gmail.com',
 '79126455359@ya.

In [ ]:
middle_time = time.time()

file = drive.CreateFile({'title': 'Анкеты без заключения', 
    "parents": [{"id": folder_names[key_short]}], 
    "mimeType": "application/vnd.google-apps.folder"})
file.Upload()
folder_id_a = file['id']

file = drive.CreateFile({'title': 'Первый эшелон', 
    "parents": [{"id": folder_names[key_short]}], 
    "mimeType": "application/vnd.google-apps.folder"})
file.Upload()
folder_id_1 = file['id']

file = drive.CreateFile({'title': 'Второй эшелон', 
    "parents": [{"id": folder_names[key_short]}], 
    "mimeType": "application/vnd.google-apps.folder"})
file.Upload()
folder_id_2 = file['id']

In [ ]:
folder_id_1 = '1fx3cRMKCIc3KCYCIyxuU5tpF4qImSj9l'
folder_id_2 = '1UX-yVLogm3frvLlF57qJoqvYVXfjub63'
folder_id_a = '1hqDqICe2Ldzf2ZasmWs7pRR0sy1jTPg-'

### Общий поиск и запись анкет

In [ ]:
c_b_1 = "Участник продемонстрировал мотивацию к переходу в предпринимательский тип деятельности и наличие потенциала к постановке и развитию предпринимательских компетенций."
c_r_1 = "Участник рекомендован к прохождению углублённой диагностики с целью уточнения оценки предпринимательских компетенций и определения потенциальной роли в предпринимательских проектах."
c_b_2 = "Участник показал наличие мотивации к переходу в предпринимательский тип деятельности, однако не продемонстрировал наличие потенциала к постановке и развитию предпринимательских компетенций."
c_r_2 = "Участник рекомендован к прохождению углублённой диагностики с целью проверки наличия потенциала к постановке и развитию предпринимательских компетенций и определения пригодности к участию в предпринимательских проектах."

In [ ]:
import matplotlib
import warnings

warnings.filterwarnings("ignore")
matplotlib.use('Agg')

keys = re.findall('{{(.+)}}', docx2txt.process(template))
keys.append('q1videoaudiourls')
result = []

for email in tqdm(emails[:84]):
    count = 0
    doc = DocxTemplate(template)
    doc_a = DocxTemplate(template_a)
    context = {}
    for df in dfs:
        mail = email_col(df)
        df[mail] = df[mail].str.lower().str.replace(' ', '')
        for key in set(df.columns.to_list()).intersection(keys):
            try:
                context[key] = max(df[df[mail] == email][key], key=len)
                if key in ['score_test', 'time']:
                    context[key] = df[df[mail] == email].groupby([mail])[key].apply('  \n  '.join)[0]
                elif key == 'q1videoaudiourls':
                    count += 1
                    context['q' + str(count)] = max(df[df[mail] == email][key], key=len)
                elif key == 'photo':
                    #doc1 = DocxTemplate(template)
                    #context['photo'] = screen_update(key, email, 'image' + key + '.png', context, doc, 7)
                    try:
                        context['photo'] = screen_update(key, email, 'image' + key + '.png', context, doc, 7)
                        #doc1.render(context)
                    except:
                        context[key] = max(df[df[mail] == email][key], key=len)
                        pass
                elif key == 'question':
                    context[key] = df[df[mail] == email].groupby([mail])[key].apply('    \n    '.join)[0]
            except:
                pass
    try:
        last, name, patronymic = context['name'].split()
        #context['games'] = ", ".join(up_to_root(last, name, root_id))
    except:
        pass
    context['name'] = max(df0[df0['ma_email'] == email]['name'], key=len)
    context['name'] = max(df0[df0['ma_email'] == email]['name_2'], key=len) + ' ' + context['name'] + ' ' + max(df0[df0['ma_email'] == email]['name_3'], key=len)
    context['email'] = email
    context['city'] = re.findall('ship-(.+)/test', max(df0[df0['ma_email'] == email]['referer'], key=len))[0]
    context['novel_score'], context['novel_translated'], context['novel_last'], context['novel_branch'], novel_numbers = novella_insert(email, data, rules)
    cases_pizza_numbers = score_cases_pizza(context)
    if novel_numbers != '-':
        context['novel_pizza'] = web(doc, novel_numbers+cases_pizza_numbers)
    else:
        context['novel_pizza'] = web(doc, cases_pizza_numbers)
        
        ###GLEB
    """if email in test_new[0].values:
        print(email)
        val = max(test_new[test_new[0]==email][1].values)
        print(val)
        try:
            print('+', context['score'],context['score_test'])
        except:
            context['score'] = str(val)
            context['score_test'] = str(val)
            print('=', context['score'],context['score_test'])"""
        ###GLEB
        
    context['score'] = primary_point(context)
    context.update({key: 'пропуск' for key in keys if key not in context or context[key] == ''})
    #if context['photo'] == 'пропуск':
    context['photo'] = InlineImage(doc, photo_default, Cm(7))
    if context['check'] == 'yes':
        context['check'] = 'ДА'
    result.append([context['name'], context['email'], context['city'], context['univer'], context['phone'], context['socialmedia'], context['score']])
    
    
    if context['score'] >= 40:
        context['conc_b'] = c_b_1
        context['conc_r'] = c_r_1
        filepath = 'Вывод/' + context['city'] + '/1/' + context['name'] + '.docx'
        doc.render(context)
        doc.save(filepath)
        print(filepath, context['score'], '1')
        drive_write(context['name'], folder_id_1, filepath)
    elif context['score'] > 0:
        context['conc_b'] = c_b_2
        context['conc_r'] = c_r_2
        filepath = 'Вывод/' + context['city'] + '/2/' + context['name'] + '.docx'
        print(filepath, context['score'], '2')
        doc.render(context)
        doc.save(filepath)
        drive_write(context['name'], folder_id_2, filepath)
    
    """
    doc.render(context)
    filepath = 'Вывод/' + context['name'] + '.docx'
    print(filepath, context['score'])
    doc.save(filepath)
    """
    context_a = context.copy()
    if novel_numbers != '-':
        context_a['novel_pizza'] = web(doc_a, novel_numbers+cases_pizza_numbers)
    else:
        context_a['novel_pizza'] = web(doc_a, cases_pizza_numbers)
    context_a['photo'] = InlineImage(doc_a, photo_default, Cm(7))
    doc_a.render(context_a)
    doc_a.save('Вывод/' + context_a['city'] + '/' + context_a['name'] + '.docx')
    
    #drive_write(context['name'], folder_names[context['city']], filepath)
    drive_write(context_a['name'], folder_id_a, 'Вывод/' + context_a['city'] + '/' + context_a['name'] + '.docx')

  1%|▉                                                                                  | 1/84 [00:07<10:52,  7.86s/it]

Вывод/urfu/2/Ковальчук Дмитрий Сергеевич.docx 11 2


  2%|█▉                                                                                 | 2/84 [00:14<09:51,  7.22s/it]

Вывод/urfu/2/Нелюбина Анна Андреевна.docx 11 2


  6%|████▉                                                                              | 5/84 [00:29<06:40,  5.08s/it]

...

 94%|█████████████████████████████████████████████████████████████████████████████     | 79/84 [09:09<00:28,  5.73s/it]

Вывод/urfu/2/Мамаев Глеб Андреевич.docx 10 2


 96%|███████████████████████████████████████████████████████████████████████████████   | 81/84 [09:20<00:16,  5.43s/it]

Вывод/urfu/2/Янин Никита Денисович.docx 10 2


100%|██████████████████████████████████████████████████████████████████████████████████| 84/84 [09:34<00:00,  6.84s/it]


### Первичный балл (печать)

In [ ]:
result_title = "Первичный балл " + datetime.datetime.now().strftime("%Y-%m-%d_%H-%M")
result_pts = pd.DataFrame(result)
result_pts.sort_values(6, ascending = False, inplace = True)
result_pts.rename(columns={0: 'ФИО', 1: 'Почта', 2: 'Город', 3: 'Курс', 4: 'Телефон', 5: 'Сеть', 6: 'Первичный балл'}, inplace = True)
result_pts.insert(0, "№", range(1, len(result_pts) + 1))
result_pts.to_excel("Вывод/{}.xlsx".format(result_title))
#print_df(result_pts, "Вывод/{}.docx".format(result_title))
#drive_write(result_title, folder_root_id, "Вывод/" + result_title, replace = False)

In [ ]:
import xlsxwriter 
qq = 'Вывод/'+result_title+'.xlsx'
workbook = xlsxwriter.Workbook('hyperlink.xlsx')
worksheet = workbook.add_worksheet('Hyperlinks')
df = pd.read_excel(qq)
df.insert(5, 'Факультет', df['Почта'].apply(lambda x: df0[df0['ma_email'] == x]['univer_2'].values[0]))
df.insert(2, 'Фамилия', df['Почта'].apply(lambda x: df0[df0['ma_email'] == x]['name_2'].values[0]))
df.insert(3, 'Имя', df['Почта'].apply(lambda x: df0[df0['ma_email'] == x]['name'].values[0]))
df.insert(4, 'Отчество', df['Почта'].apply(lambda x: df0[df0['ma_email'] == x]['name_3'].values[0]))
try:
    cities = df['Город'].unique()
except:
    pass
link_dict = {}

for city_id in [folder_id_1, folder_id_2]:
    for file in drive.ListFile({'q': " '%s' in parents and trashed = FALSE" % city_id}).GetList():
        link_dict.update({file['title']: file['alternateLink']})
"""
for city in cities:
    city_id = drive.ListFile({'q': "title = '%s'" % city}).GetList()[0]['id']
    for file in drive.ListFile({'q': " '%s' in parents and trashed = FALSE" % city_id}).GetList():
        link_dict.update({file['title']: file['alternateLink']})
"""

df['Ссылка'] = df['ФИО'].copy()
links = df['ФИО'].apply(lambda x: link_dict[x] if x in link_dict else 'Error').to_list()
df['Ссылка'] = links
df.drop(columns=['Unnamed: 0', 'ФИО'], inplace=True)
df.to_excel('Вывод/'+context['city']+'/Анкетосики '+context['city']+'.xlsx', index=False)

### Запись с ссылками (внимание)

In [ ]:
a=print_df_linked('Вывод/'+result_title+'.docx', 'Первичный балл + ссылки.docx')
a

PackageNotFoundError: Package not found at 'Вывод/Первичный балл 2022-04-07_18-31.docx'

### Временные затраты

In [ ]:
print("\nСоздано %s отчетов" % len(emails))
print("Время работы: %s секунд" % round(time.time() - start_time))
print("Cкачивание и подключение данных: %s секунд" % round(middle_time - start_time))
print("Заполнение анкет: %s секунд" % round(time.time() - middle_time))
print("Средняя скорость ~ %s секунд на анкету" % round((time.time() - middle_time) / len(emails), 3))


Создано 567 отчетов
Время работы: 2697 секунд
Cкачивание и подключение данных: 1470 секунд
Заполнение анкет: 1227 секунд
Средняя скорость ~ 2.164 секунд на анкету


### Пользовательский интерфейс (Запуск Анкеты)

import PySimpleGUI as sg
import pandas as pd

layout = [
    [sg.Text('Таблица 1'), sg.InputText('Вопрос от Ивана Федосеева'), sg.FileBrowse('Указать путь')],
    [sg.Text('Таблица 2'), sg.InputText('Вопрос от Ольги Орловой'), sg.FileBrowse('Указать путь')],
    [sg.Text('Таблица 3'), sg.InputText('Вопрос от Катерины Ходуновой'), sg.FileBrowse('Указать путь')],
    [sg.Text('Таблица 4'), sg.InputText('Вопрос от Олега Лысака'), sg.FileBrowse('Указать путь')],
    [sg.Text('Таблица 5'), sg.InputText('Вопрос от Ивана Бородина'), sg.FileBrowse('Указать путь')],
    [sg.Text('Таблица 6'), sg.InputText('Тестирование 21 вопрос'), sg.FileBrowse('Указать путь')],
    [sg.Output(size=(88, 18))],
    [sg.Submit('Запустить'), sg.Cancel('Выйти')]
]


email = 'bgdshv.a@gmail.com'
context = {}
window = sg.Window('Анкеты + Первичный Балл', layout)
while True:
    event, values = window.read()
    if event in (None, 'Exit', 'Выйти'):
        break
    if event == 'Запустить':
        count = 0
        tables_input = [values[num] for num in range(6)]
        print(tables_input)
        ank()
window.close()